<a href="https://colab.research.google.com/github/nanglo123/German-Traffic-Sign-CV/blob/main/Traffic_Sign_Colab_for_GitHub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import all libraries and frameworks needed

In [2]:
!pip install torchmetrics
!pip install torchinfo 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import zipfile
import numpy as np
import pandas as pd
import os 
import csv
import shutil 
import os
import time


import torchmetrics
import torch
from torch import nn 
from torch.utils.data import Dataset,DataLoader, Subset

import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms

from glob import glob
from PIL import Image
from google.colab import files 
from pathlib import Path
from tqdm.auto import tqdm

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

###Download and unzip data

In [5]:
#upload kaggle.json file 
files.upload()

!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Saving kaggle.json to kaggle (1).json
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
#Create data folder and download the dataset from Kaggle 
data_folder = Path('Data')
if data_folder.exists():
  print('Folder Created')
else:
  data_folder.mkdir(parents=True,exist_ok=True)


!kaggle datasets download meowmeowmeowmeowmeow/gtsrb-german-traffic-sign -p 'Data'

100% 611M/612M [00:16<00:00, 44.3MB/s]
100% 612M/612M [00:16<00:00, 39.3MB/s]


In [7]:
#Unzip downloaded zip file 
with zipfile.ZipFile('/content/Data/gtsrb-german-traffic-sign.zip', "r") as zip_ref:
  print(f"Unzipping traffic {data_folder}")
  zip_ref.extractall('/content/Data/') 


Unzipping traffic Data


###Format the Data

In [15]:
#Create train and test directories paths
#Currently the downloaded dataset does not have folders for each class for test data (i.e. all test images from all classes are in 1 folder)
  #Need in format specified below to make compatiable with ImageFolder object from torchvision library 
    #Test
      #Class 1
        #Picture of Class 1
      #Class 2
        #Picture of Class 2 
  #Have to fix the problem in later cells 

train_dir = data_folder / 'Train'
test_dir_with_label = data_folder/'testwithfolders'


#Create test directory with sub directories for each class, the Train directory from the dataset has sub directories for each class so it does not need to be created
if not test_dir_with_label.exists():
  test_dir_with_label.mkdir(parents=True,exist_ok=True)


#Create list of all class names from the already established train directory (e.g. 1,2,3,4,..., 43)
class_names = list(os.scandir(train_dir))


#for each class name in the list of class names
  #Create a class folder in the test directory with the title of the sub-directory/folder equal to the class name  
for dir_entry in class_names:
  test_class_folders = test_dir_with_label / str(dir_entry.name)
  if not test_class_folders.exists():
    test_class_folders.mkdir(parents=True,exist_ok=True)


In [9]:
#read 'test.csv' meta data file
  #populate the list 'list_id' with the assigned class for each image in the test dataset listed in the 'test.csv' file by turning the csv file into a pd dataframe
  #the 'test.csv' file lists the meta data for each image in numerical order (i.e. information listed for the image '0000.png' listed first and then info for photo '0001.png')
    

list_of_class_id = None
with open(data_folder / 'Test.csv', newline='') as csvfile:
  df = pd.read_csv(csvfile)
  list_of_class_id = list(df['ClassId'])



In [10]:
#Create pathway to the existing unformatted Test folder with all test images
#Create a list containing path to all test images sorted in numerical order
  #(i.e. first element of list is image '0000.png' and second element is '0001.png')
filenames = None
test_dir_no_labels = data_folder/'Test'
image_path_list = sorted(list(test_dir_no_labels.glob("*.png")))


#Get the class number (e.g. 5,7,23)
#Create pathway to correct class folder in new test directory
#Move the path of the image in 'image_path_list' to the correct class folder in new test directory

#NOTE: Can cast PosixPath objects as strings just get the name of a directory
try:
  for pic,i in zip(image_path_list,range(len(image_path_list))):
    class_label = list_of_class_id[i]
    test_picture_folder_with_label = test_dir_with_label / str(class_label)
    shutil.move(str(pic),test_picture_folder_with_label)
except:
  print('Already moved that picture to the correct class folder in the new test directory')
    

In [12]:
#Create dictionary containing all class number/label to class name key-value pairings 
classes_dict = { 0:'Speed limit (20kmh)',
            1:'Speed limit (30kmh)', 
            2:'Speed limit (50kmh)', 
            3:'Speed limit (60kmh)', 
            4:'Speed limit (70kmh)', 
            5:'Speed limit (80kmh)', 
            6:'End of speed limit (80kmh)', 
            7:'Speed limit (100kmh)', 
            8:'Speed limit (120kmh)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice or snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }



In [22]:
#Rename class folders in train and test directories with the class label instead
  #(e.g. 1 -> Straight Ahead Sign)

#This turns the ordering of the class files in both the test and train directory into alphabetically ordered ones based on class label instead of class number
  #(e.g. 0 -> 1 -> 2 could now become 2-> 0 -> 1 if class label for class 2 is first alphabetically)
  #Have to keep this in mind when preparing list of class labels for the Gradio app because we cannot just read off the folder/class label names in the train or test directory
    #Have to reorder to match class folder order when data first imported before renaming later
for train_class_dir in os.listdir(train_dir):
  try:
    train_class_name = classes_dict[int(train_class_dir)]
  except ValueError:
    continue
  os.rename(os.path.join(train_dir, train_class_dir), os.path.join(train_dir, train_class_name))

for class_dir in os.listdir(test_dir_with_label):
  try:
    class_name = classes_dict[int(class_dir)]
  except ValueError:
    continue
  os.rename(os.path.join(test_dir_with_label, class_dir), os.path.join(test_dir_with_label, class_name))

In [11]:
#Remove unecessary/duplicate/empty files and folders after all data is properly formatted 
!rm -r Data/Meta
!rm -r Data/test
!rm -r Data/train
!rm -r Data/meta
!rm Data/gtsrb-german-traffic-sign.zip

#Can remove this after moving all test images from this directory into proper test directory 
!rm -r Data/Test


#Rename 'testwithfolders' directory containing properly formatted test data
!mv Data/testwithfolders Data/Test

In [25]:
#Create path variable for new Test directory
test_dir = data_folder/'Test'

###Create train and test loaders

In [27]:
#Create train and test datasets and loaders with fixed BATCH_SIZE
BATCH_SIZE = 32
train_transforms = transforms.Compose([
    transforms.Resize(size=(32,32)),
    transforms.ToTensor(),
  
])
test_transforms = transforms.Compose([
    transforms.Resize(size=(32,32)),
    transforms.ToTensor()
])



train_set = ImageFolder(root=train_dir,transform=train_transforms)
train_loader = DataLoader(dataset=train_set,batch_size=BATCH_SIZE,shuffle=True)

test_set = ImageFolder(root=test_dir,transform=test_transforms)
test_loader = DataLoader(dataset=test_set,batch_size=BATCH_SIZE, shuffle=False)



###Define the neural network class

In [28]:

class CNNTraffic(nn.Module):
    def __init__(self,input_shape:int,output_shape:int):
        super().__init__()

        self.layer1 = nn.Sequential(
          nn.Conv2d(in_channels = input_shape, out_channels = 32, kernel_size=5, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=5, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2),
          nn.Dropout(p=.25)
          )

        self.layer2 = nn.Sequential(
          nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2),
          nn.Dropout(p=.25))


        self.fc = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=3136,out_features=256),
          nn.Dropout(p=.5),
          nn.Linear(in_features=256,out_features=output_shape))


    def forward(self,x):
      return self.fc(self.layer2(self.layer1(x)))

In [37]:
#Create the model with correct number of input_shape and output_shape with its loss function and optimzier as well 

sample,label =  next(iter(train_set))
color_channels = sample.shape[0]

output_length = len(train_set.classes)


m2 = CNNTraffic(input_shape = color_channels, output_shape=output_length).to(device)
opt = torch.optim.Adam(params=m2.parameters(), lr=.001)
loss_fn = nn.CrossEntropyLoss()

In [29]:
#Create path to save model 
models_save_path = Path('Saved Models')
if not models_save_path.exists():
  models_save_path.mkdir(parents=True,exist_ok=True)



###Define train and test steps

In [39]:
def train_step(model:nn.Module,
               opt:torch.optim,
               loss_fn:nn.Module,
               train_dataloader:torch.utils.data.DataLoader,
               device:str):
  
  model.train()
  train_loss,train_acc = 0,0
  for X,y in train_dataloader:
    X = X.to(device)
    y = y.to(device)

    train_logit = model(X)


    #these loss and acc metrics are being computed per batch in loop, not per sample
    loss = loss_fn(train_logit,y)

    train_loss += loss.item()

    opt.zero_grad()

    loss.backward()
    opt.step()

    train_pred_labels = torch.argmax(torch.softmax(train_logit,dim=1),dim=1)

    train_acc += (train_pred_labels==y).sum().item()/len(y)

  #get average loss and acc per batch
  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  return train_loss,train_acc

def test_step(model:nn.Module,
               loss_fn:nn.Module,
               test_dataloader:torch.utils.data.DataLoader,
               device:str):
  model.eval()
  test_loss,test_acc = 0,0
  with torch.inference_mode():
    for X,y in test_dataloader:
      X = X.to(device)
      y = y.to(device)

      test_logit = model(X)

      loss = loss_fn(test_logit,y)

      test_loss += loss.item()

    
      test_pred_labels = torch.argmax(torch.softmax(test_logit,dim=1),dim=1)

      test_acc += (test_pred_labels==y).sum().item()/len(y)

    test_loss /= len(test_dataloader)
    test_acc /= len(test_dataloader)
  return test_loss,test_acc

In [38]:
# 1. Take in various parameters required for training and test steps
def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          device = device,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):
    
    # 2. Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []
    }
    
    # 3. Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                           train_dataloader=train_loader,
                                           loss_fn=loss_fn,
                                           opt=optimizer,
                                           device = device)
        
        test_loss, test_acc = test_step(model=model,
            test_dataloader=test_loader,
            loss_fn=loss_fn,
            device = device)
        
        # 4. Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # 5. Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        
        model_save_path = 'Saved Models/VGGNETINSPIRED_replica_epoch' +str(epoch+1) + '.pth'
        torch.save(model.state_dict(), model_save_path)
    # 6. Return the filled results at the end of the epochs
    return results



###Begin training and record metrics such as train/test accuracy and loss

In [41]:
start = time.time()
results = train(model=m2,train_dataloader = train_loader,test_dataloader=test_loader,
      optimizer = opt, loss_fn=loss_fn,epochs=25, device=device)
end = time.time()
print(f' Time took  {(end - start)/60}  minutes')



  0%|          | 0/25 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.9995 | train_acc: 0.7080 | test_loss: 0.2981 | test_acc: 0.9187
Epoch: 2 | train_loss: 0.1855 | train_acc: 0.9451 | test_loss: 0.2293 | test_acc: 0.9350


KeyboardInterrupt: ignored

###Create files needed for Deployment on Hugging Face using Gradio###

In [42]:
deployment_path = Path('Deployment')
deployment_path.mkdir(parents=True,exist_ok=True)

In [44]:
%%writefile Deployment/class_names.txt
Ahead only
Beware of ice or snow
Bicycles crossing
Bumpy road
Children crossing
Dangerous curve left
Dangerous curve right
Double curve
End no passing veh > 3.5 tons
End of no passing
End of speed limit (80kmh)
End speed + passing limits
General caution
Go straight or left
Go straight or right
Keep left
Keep right
No entry
No passing
No passing veh over 3.5 tons
No vehicles
Pedestrians
Priority road
Right-of-way at intersection
Road narrows on the right
Road work
Roundabout mandatory
Slippery road
Speed limit (100kmh)
Speed limit (120kmh)
Speed limit (20kmh)
Speed limit (30kmh)
Speed limit (50kmh)
Speed limit (60kmh)
Speed limit (70kmh)
Speed limit (80kmh)
Stop
Traffic signals
Turn left ahead
Turn right ahead
Veh > 3.5 tons prohibited
Wild animals crossing
Yield

Writing Deployment/class_names.txt


In [43]:
%%writefile Deployment/requirements.txt
torch==1.12.0
torchvision==0.13.0
gradio==3.1.4


Writing Deployment/requirements.txt


In [45]:
%%writefile Deployment/app.py 

### 1. Imports and class names setup ### 
import gradio as gr
import os
import torch


from timeit import default_timer as timer
from typing import Tuple, Dict
from create_model import create_CNN
import torchvision
from torchvision import transforms
# Setup class names
with open("class_names.txt", "r") as f: # reading them in from class_names.txt
    class_names = [food_name.strip() for food_name in  f.readlines()]
    
### 2. Model and transforms preparation ###    

# Create model
model = create_CNN()

# Load saved weights
model.load_state_dict(
    torch.load(
        f="model_adam_epoch25.pth",
        map_location=torch.device("cpu"),  # load to CPU because gpu not guaranteed
    )
)

### 3. Predict function ###

# Create predict function
def predict(img) -> Tuple[Dict, float]:
    """Transforms and performs a prediction on img and returns prediction and time taken.
    """
    # Start the timer
    start_time = timer()
    t = transforms.Compose([
        transforms.Resize(size=(32,32)),
        transforms.ToTensor()
        ])

    # Transform the target image and add a batch dimension
    img = t(img).unsqueeze(0)
    
    # Put model into evaluation mode and turn on inference mode
    model.eval()
    with torch.inference_mode():
        # Pass the transformed image through the model and turn the prediction logits into prediction probabilities
        pred_probs = torch.softmax(model(img), dim=1)
    
    # Create a prediction label and prediction probability dictionary for each prediction class (this is the required format for Gradio's output parameter)
    pred_labels_and_probs = {class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))}
    
    # Calculate the prediction time
    pred_time = round(timer() - start_time, 5)
    
    # Return the prediction dictionary and prediction time 
    return pred_labels_and_probs, pred_time

### 4. Gradio app ###

# Create title, description and article strings
title = "GTSRB - German Traffic Sign Recognition by Tenzin Nanglo"
description = "CNN created for the GTSRB Dataset, achieved 98% test accuracy"
article = "Created a 3 block CNN consisting of two convolutional blocks and one classifier block. The model was trained on 25 epochs and a batch size of 32 using an Adam Optimizer with a learning rate of .001. The design of the CNN architecture was inspired by other projects conducted on this dataset. There were no additional preprocessing procedures done on the data besides resizing them into 32x32 images."

# Create examples list from "examples/" directory
example_list = [["examples/" + example] for example in os.listdir("examples")]

# Create Gradio interface 
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Label(num_top_classes=5, label="Predictions"),
        gr.Number(label="Prediction time (s)"),
    ],
    examples=example_list,
    title=title,
    description=description,
    article=article,
)

# Launch the app!
demo.launch()

Writing Deployment/app.py


In [46]:
%%writefile Deployment/model_class.py 
from torch import nn

class CNNTraffic(nn.Module):
    def __init__(self,input_shape:int,output_shape:int):
        super().__init__()

        self.layer1 = nn.Sequential(
          nn.Conv2d(in_channels = input_shape, out_channels = 32, kernel_size=5, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size=5, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2),
          nn.Dropout(p=.25)
          )

        self.layer2 = nn.Sequential(
          nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size = 2),
          nn.Dropout(p=.25))


        self.fc = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=3136,out_features=256),
          nn.Dropout(p=.5),
          nn.Linear(in_features=256,out_features=output_shape))


    def forward(self,x):
        return self.fc(self.layer2(self.layer1(x)))

Writing Deployment/model_class.py


In [47]:
%%writefile Deployment/create_model.py
import torch
import torchvision
from model_class import CNNTraffic
from torch import nn
from torchvision import transforms

def create_CNN(seed:int=42):

    model = CNNTraffic(input_shape=3,output_shape=43)

    
    for param in model.parameters():
        param.requires_grad = False
    
    
    
    return model


Writing Deployment/create_model.py


In [56]:
#Make an examples folder and add three images from test set to examples
examples_path = deployment_path / 'examples'
examples_path.mkdir(parents=True, exist_ok=True)

examples = [Path('/content/Data/Test/Ahead only/00009.png'),
                            Path('/content/Data/Test/Beware of ice or snow/00041.png'),
                            Path('/content/Data/Test/Beware of ice or snow/00443.png')]

#Copy the three random images to the examples directory
for example in examples:
    destination = examples_path / example.name
    print(f"[INFO] Copying {example} to {destination}")
    shutil.copy2(src=example, dst=destination)

[INFO] Copying /content/Data/Test/Ahead only/00009.png to Deployment/examples/00009.png
[INFO] Copying /content/Data/Test/Beware of ice or snow/00041.png to Deployment/examples/00041.png
[INFO] Copying /content/Data/Test/Beware of ice or snow/00443.png to Deployment/examples/00443.png


In [57]:
#Zip all files related to deployment and download the zip file 
!cd Deployment/ && zip -r Deployment * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"
files.download("Deployment/Deployment.zip")


updating: app.py (deflated 54%)
updating: class_names.txt (deflated 53%)
updating: create_model.py (deflated 45%)
updating: model_class.py (deflated 70%)
updating: requirements.txt (deflated 4%)
  adding: examples/ (stored 0%)
  adding: examples/00443.png (stored 0%)
  adding: examples/00041.png (stored 0%)
  adding: examples/00009.png (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>